# Big Data y Machine Learning para clasificación de galaxias

- [Descarga de datos](#data)
    - [Dataset de parámetros e instrumentación](#data1)
    - [Dataset GalaxyZoo](#data2)
    - [Dataset de imágenes](#data3)
- [Almacenamiento HDFS](#hdfs)

    
<div id='xx' />

<div id='data' />

## Descarga de datos

Primero que nada debemos descargar nuestro datos en el contenedor y, posteriormente, almacenarlos en HDFS.

Descargaremos varios ficheros `.csv` que consisten en un sub-conjunto de datos de la web del proyecto [SDSS](https://www.sdss.org/) almacenados en un repositorio de la UAB. En concreto descargaremos los datos relativos a la *Data Realease 14*: [link1](https://www.sdss.org/dr14/) y [link2](http://skyserver.sdss.org/dr14/en/tools/toolshome.aspx). Éstos datos contendrán información delativa al brillo del objeto, características del telescopio, etc.

Las imágenes de galaxias tomadas para este proyecto son descargadas como imágenes `png` de 64x64 píxeles en color en base a un script para luego ser procesadas y almacenadas en un fichero.

- [SDSS DR14 Finding Chart Tool](https://skyserver.sdss.org/dr14/en/tools/chart/chartinfo.aspx)
- [SDSS Science Archive Server (SAS)](https://dr14.sdss.org/comingsoon/imaging/field/search)

---

<img src="../images/galaxias.png">

---

Las imágenes que descargaremos serán aquellas que corresponden al dataset del proyecto *GalaxyZoo* y por lo tanto tendremos la certeza de que contamos con una clasificación del tipo de galaxia.

<div id='data1' />

#### Dataset: parámetros e instrumentación

Mediante estas sencillas instrucciones descargaremos los datos (trabajaremos dentro del directorio `/home/cloudera/`):

`
[container]$ mkdir -p /home/cloudera/galaxias/data
[container]$ cd /home/cloudera/galaxias/data
[container]$ wget https://nebula.uab.cat/share/proxy/alfresco-noauth/api/internal/shared/node/7mRDxMhySJu0OMG0drr1MA/content/SDSS_parametros.zip
[container]$ unzip SDSS_parametros.zip
[container]$ tree /home/cloudera/galaxias/data
`

~~~
galaxias/data/
├── SDSS_PhotObj.csv
└── SDSS_SpecObj.csv
~~~

---

<div id='data2' />

#### Dataset: GalaxyZoo

El [dataset](https://data.galaxyzoo.org/) de GalaxyZoo representa el conjunto de datos principal de este proyecto por dos motivos. En primer lugar es el que contiene la correlación entre el `id` del objeto (nuestra imagen) y la clasificación de los usuarios. Y en segundo lugar es el que utilizaremos para seleccionar las imágenes que vamos a descargar.

`
[container]$ cd /home/cloudera/galaxias/data
[container]$ wget http://galaxy-zoo-1.s3.amazonaws.com/GalaxyZoo1_DR_table2.csv.zip
[container]$ unzip GalaxyZoo1_DR_table2.csv.zip
[container]$ tree /home/cloudera/galaxias/data
`

~~~
galaxias/data/
├── GalaxyZoo1_DR_table2.csv
├── SDSS_PhotObj.csv
└── SDSS_SpecObj.csv
~~~

---

<div id='data3' />

#### Dataset: conjunto de imágenes

Desarrollaremos un script para la descarga de imágenes que posteriormente utilizaremos para entrenar nuestros modelos de clasificación.

La mayoría de algoritmos de machine learning necesitan que los datos estén organizados de forma tabular, con las observaciones (imágenes en nuestro caso) distribuidas por filas y los parámetros (píxeles) de cada observación en columnas. Así pues, convertiremos las imágenes en un vector de `64x64 = 4096` posiciones en base a los siguientes pasos:

- Convertir las imágenes en color a una imagen en blanco y negro. Cada uno de los píxeles de la imagen contiene 3 valores entre 0 y 255 correspondientes a los 3 canales RGB (R=Rojo, G=Verde, B=azul). Podríamos decir que cada imagen está compuesta por tres matrices `64x64: MR, MG, MB` correspondientes a los canales R, G, B respectivamente. Aplicaremos la siguiente formula para fusionar las tres matrices en una sola matriz en tono de grises: `M = 0.2989 * MR + 0.5870 * MG + 0.1140 * MB`


- Aplanamos la matriz `M` a un vector, esto es, reorganizamos los valores de la matriz `M` de dimensiones `64x64` a un vetor de longitud `4096`.


- Algunos algoritmos funcionan mejor si todas las variables se encuentran dentro del mismo rango de valores, típicamente `[-1, 1] o [0, 1]`. Por ello normalizamos los valores del vector dividiendolos entre 255.


- Para poder entrenar un algoritmo de clasificación también necesitaremos los *targets* de las observaciones. Éstas se obtienen de las clasificaciones del dataset de GalaxyZoo y tienen los siguientes valores:


    - target = 0 -> clase = incierto
    - target = 1 -> clase = elíptica
    - target = 2 -> clase = espiral
    
- Juntamos los dos conjuntos de datos en un solo fichero que contiene tanto los *features* como el *target*. Este conjunto de datos tiene los campos:


    - drobjid: identificador del objeto.
    - target: clase del objeto.
    - F0 a F4095: correspondiente al vector de 4095 atributos de la imagen.
    
El desarrollo de este apartado la llevaremos a cabo en la segunda parte de este trabajo.

<div id='hdfs' />

## Almacenamiento HDFS

Para comunicarse con el sistema distribuido de ficheros de HADOOP (HDFS), existen una serie de comandos de shell proporcionados por el mismo sistema. Lo primero que vamos a hacer es crear un directorio en el HDFS para cada uno de los ficheros que posteriormente vamos a analizar. Para la creación de un directorio en HDFS utilizamos el siguiente comando:

---

`[container]$ hdfs dfs -mkdir /user/cloudera/galaxias
 mkdir: Permission denied: user=root, access=WRITE, inode="/user/cloudera":cloudera:cloudera:drwxr-xr-x
`

---

Vemos que no podemos escribir sobre ese destino en el HDFS como usuario 'root' ya que pertenece al usuario 'cloudera'. Además no es muy aconsejable trabajar con 'root' en HDFS. En definitiva cambiamos a usuario 'impala' y nuestro directorio destino (posteriormente veremos el motivo de esta elección):

---

`
[container]$ su impala
[container]$ pwd
/home/cloudera/galaxias
[container]$ hdfs dfs -mkdir /user/hive/galaxias
[container]$ hdfs dfs -ls /user/hive
Found 2 items
drwxr-xr-x   - impala supergroup          0 2020-05-23 13:25 /user/hive/galaxias
drwxrwxrwx   - hive   supergroup          0 2020-05-22 22:07 /user/hive/warehouse
`

---

Creamos un directorio para cada uno de los archivos de datos dentro de HDFS de igual manera que hicimos en el caso anterior:

---

`
[container]$ hdfs dfs -mkdir /user/hive/galaxias/SDSS_PhotObj
[container]$ hdfs dfs -mkdir /user/hive/galaxias/SDSS_SpecObj
`

---

El motivo de haber creado un directorio para cada fichero se explicará más adelante a la hora de modelizar los datos.

A continuación, para realizar la carga o ingestión de los datos desde el directorio local hasta el directorio que acabamos de crear en el HDFS usamos el comando `put`:

---

`
[container]$ hdfs dfs -put data/SDSS_PhotObj.csv /user/hive/galaxias/SDSS_PhotObj/SDSS_PhotObj.csv
[container]$ hdfs dfs -put data/SDSS_SpecObj.csv /user/hive/galaxias/SDSS_SpecObj/SDSS_SpecObj.csv
`

---

Y podemos comprobar que ambos ficheros se han creado correctamente:

---

`
[container]$ hdfs dfs -ls /user/hive/galaxias/SDSS_PhotObj/
Found 1 items
-rw-r--r--   1 impala supergroup  1248036 /user/hive/galaxias/SDSS_PhotObj/SDSS_PhotObj.csv
[container]$ hdfs dfs -ls /user/hive/galaxias/SDSS_SpecObj/
Found 1 items
-rw-r--r--   1 impala supergroup  1248036 /user/hive/galaxias/SDSS_SpecObj/SDSS_SpecObj.csv
`

---

Esto mismo lo podemos comprobar accediendo al NODEMANAGER vía web a través de la url http://localhost:50070 y seleccionando *'Browse the file system'*. Veamos un ejemplo:

---

<img src="../images/nodemanager_1.png">

---

Navegamos dentro del directorio de archivos hasta encontrar nuestros datos recién almacenados.

---

<img src="../images/nodemanager_2.png">

Para copiar algún fichero desde el HDFS al directorio local de nuestro contenedor entonces tendríamos que usar el comando `get`, por ejemplo:

---

`
[container]$ hdfs dfs -get /user/hive/galaxias/SDSS_SpecObj/SDSS_SpecObj.csv ~/galaxias/data/SDSS_SpecObj.csv
`

---

Podemos ver el tamaño de los archivos transferidos a HDFS con el siguiente comando:

---

`
[container]$ hdfs dfs -count -h /user/hive/galaxias/SDSS_PhotObj/SDSS_PhotObj.csv
           0            1              2.8 M /user/hive/galaxias/SDSS_PhotObj/SDSS_PhotObj.csv
[container]$ hdfs dfs -count -h /user/hive/galaxias/SDSS_SpecObj/SDSS_SpecObj.csv
           0            1              1.2 M /user/hive/galaxias/SDSS_SpecObj/SDSS_SpecObj.csv
`

---